In [2]:
import pandas as pd
import numpy as np

## Wczytanie słownika i wyfiltrowanie interesujących danych

In [3]:
df = pd.read_csv('./plwordnet_4_2/słownik_anotacji_emocjonlanej.csv')

In [4]:
nacechowania = pd.DataFrame(df.stopien_nacechowania).drop_duplicates()['stopien_nacechowania'].tolist()
nacechowania.remove(np.nan)
nacechowania

['- s', 'amb', '- m', '+ s', '+ m', '- s ']

In [5]:
df = df[df.stopien_nacechowania.isin(nacechowania)]
df

,lemat,wariant,czesc_mowy,jednostka_nacechowana,emocje,wartosciowane,stopien_nacechowania,przyklad1,przyklad2
2,abstrakcyjny,1,przymiotnik,Nie,wstręt;złość;zaskoczenie,niewiedza;błąd;nieużyteczność,- s,"Jej propozycje są zupełnie abstrakcyjne, nie d...",NaN
3,abstrakcyjny,1,przymiotnik,Nie,wstręt;złość,niewiedza;błąd;nieużyteczność,- s,"Nie wiem, jak powinnam powiedzieć mężowi, że j...",NaN
10,adorator,1,rzeczownik,Nie,złość;radość,szczęście;błąd,amb,Jola była piękną kobietą i miała wielu adorato...,Mój adorator był bardzo namolny.
11,adorator,1,rzeczownik,Nie,złość;cieszenie się na;radość;wstręt,szczęście;nieszczęście;błąd;dobro;nieużyteczność,amb,Franciszek był wytrwałym adoratorem Małgorzaty...,"Aktorka, nawet w rezydencji prywatnej, nie mog..."
18,afektowany,1,przymiotnik,Nie,złość;smutek,błąd;nieużyteczność,- s,Denerwował mnie jego afektowany sposób mówienia.,NaN
...,...,...,...,...,...,...,...,...,...
178505,zaciąć,11,czasownik,Nie,radość;,NaN,+ s,"Perkusista zaciął takie solo, że padliśmy z za...",NaN
178508,włóczyć,7,czasownik,Nie,złość;,nieużyteczność;błąd;,- s,Czy ty musisz włóczyć zdobycze śmietnikowe do ...,NaN
178509,włóczyć,7,czasownik,Nie,NaN,nieużyteczność;,- s,Po kiego grzyba włóczysz do domu te wszystkie ...,NaN
178512,obcierać się,6,czasownik,Nie,radość;smutek;strach;,dobro;szczęście;krzywda;nieszczęście;,amb,Nie mam ochoty po raz kolejny obcierać się o ś...,"Widziałem, że obcieram się o zwycięstwo."


In [6]:
word_sentiment_df = df[['lemat','stopien_nacechowania']].drop_duplicates()
word_sentiment_df

,lemat,stopien_nacechowania
2,abstrakcyjny,- s
10,adorator,amb
18,afektowany,- s
20,afera,- m
32,agonia,- s
...,...,...
178476,zrąbać,- s
178498,nawłóczyć,- s
178502,nawłóczyć się,- s
178504,zaciąć,+ s


In [7]:
word_sentiment_dict = {}
for index, row in word_sentiment_df.iterrows():
    word_sentiment_dict[row['lemat']] = row['stopien_nacechowania']

## Wczytanie tweetów do analizy

In [8]:
# pomocnicze funkcje

def flatten_dct_(prefix, dct):
    res = {}
    for key, value in dct.items():
        if not isinstance(value, dict):
            res[prefix + key] = value
        else:
            for key2, value2 in flatten_dct_(key+".", value).items():
                res[prefix+key2] = value2
    return res

def flatten_dct(dct):
    return flatten_dct_('', dct)



#### Wczytanie tweetów do pd.DataFrame

In [9]:
import pymongo

DB_CONN_STRING = "mongodb://localhost:27017/"
DB_NAME = "poland-covid-vaccine-tweets"
COLLECTION_NAME = "tweets"

mongo_client = pymongo.MongoClient(DB_CONN_STRING)
database_name = mongo_client[DB_NAME]
data_collection = database_name[COLLECTION_NAME]

projection = {"date":1, "content":1, "user.username":1, "user.id":1, "user.followersCount":1, "user.verified":1}
tweets = [flatten_dct(tweet) for tweet in data_collection.find({},projection)]
# print(tweets[0])

df = pd.DataFrame(tweets)
df

,_id,date,content,user.username,user.id,user.verified,user.followersCount
0,1244948603606630403,2020-03-31T11:24:11+00:00,$AZN Astrazeneca PLC SEC Filing: Form 6-K http...,StockTexts,772471873539977217,False,1776
1,1244574577923182593,2020-03-30T10:37:57+00:00,$AZN Astrazeneca PLC SEC Filing: Form 6-K http...,StockTexts,772471873539977217,False,1776
2,1243835202172002304,2020-03-28T09:39:56+00:00,@jarek_jjo @ImpactCEE @300gospodarka Super! Cz...,AndrzejBarban,1217555010533429250,False,6
3,1243203045418303488,2020-03-26T15:47:58+00:00,"@Novice_trader9 Shell/BP, BHP, AstraZeneca?",sivwardo,1026477719750692864,False,31
4,1243123703732482048,2020-03-26T10:32:41+00:00,$AZN Astrazeneca PLC SEC Filing: Form 6-K http...,StockTexts,772471873539977217,False,1776
...,...,...,...,...,...,...,...
370115,1344803654352728064,2021-01-01T00:32:31+00:00,Rzeczpospolita: Trudno o rekompensatę za szkod...,Arotoko,547197601,False,402
370116,1344800670134788096,2021-01-01T00:20:39+00:00,"Kochani, dobry początek roku. @PanZolty obieca...",pjotrowicz,989641938553966595,False,5833
370117,1344799918309965830,2021-01-01T00:17:40+00:00,@LeszekMiller Prawdziwego mężczyznę rozpoznaje...,kotapczan,1323947394870972416,False,70
370118,1344799426984947714,2021-01-01T00:15:43+00:00,"@rozznet Stać, to Unię, by szczepionkę kupić. ...",Grzegorz_Red,1302918570867912706,False,791


#### Wyczyszczenie tweetów z hashtagów, cytowań, linków... 

In [10]:
import re

def clean_up_tweet(tweet):
    tweet = ' ' + tweet + ' ' #added to simplify regex below
    tweet = re.sub(r' [$#@][^ ]* | http[^ ]* ', ' ', tweet)[1:-2]
    return re.sub(r'[^\w ]', '', tweet).lower()

df['content_cleaned'] = [clean_up_tweet(tw) for tw in df['content'].values]

#### Sprowadzenie słów do podstawowej formy gramatycznej

In [11]:
from stempel import StempelStemmer
stemmer = StempelStemmer.default()

Loading: 100%|██████████████████████████████████████| 2225192/2225192 [00:03<00:00, 635980.25bytes/s]


In [12]:
def to_base_grammar_form(tweet):
    result_tweet = ''
    for word in tweet.split(' '):
        if len(word) > 0:
            try:
                result_tweet += ' ' + stemmer.stem(word)
            except TypeError as e:
                result_tweet += ' ' + word
    return result_tweet[1:]

df['content_cleaned'] = [to_base_grammar_form(tw) for tw in df['content_cleaned'].values]

## Analiza sentymentu

In [41]:
def get_sentiment(tweet):
    sentiment = 0
    f = {
        '- s' : -0.5,
        '- s ': -0.5,
        '- m' : - 1,
        '+ s' : 0.5,
        '+ m' : 1,
        'amb' : 0
    }
    for word in tweet.split(' '):
        try:
            sentiment += f[word_sentiment_dict[word]]
#             print("Word: {} Elementar sentiment: {}".format(word, f[word_sentiment_dict[word]]))
        except KeyError:
            pass
        
    return sentiment

In [46]:
df['sentiment'] = [get_sentiment(tw) for tw in df['content_cleaned'].values]

In [47]:
# tw_idx = 370038
# print(df['content'][tw_idx] + "  --> " + str(get_sentiment(df['content_cleaned'][tw_idx])))